In [2]:
import numpy as np
import pandas as pd
import shutil
from roboflow import Roboflow
import os
import re

In [3]:
def clean_up(integral=False):
    if os.path.exists('GARBAGE-CLASSIFICATION-3-2'):
        shutil.rmtree('GARBAGE-CLASSIFICATION-3-2')
    if integral:
        if os.path.exists('data'):
            shutil.rmtree('data')
    else:
        for file in os.listdir('data'):
            file = os.path.join('data', file)
            if os.path.isfile(file):
                os.remove(file)

def move_files_recursively(src,dest='data',files=None):
    if not os.path.exists(dest):
        os.makedirs(dest)
    files_to_move = []
    if files is None:
        for root, dirs, files in os.walk(src):
            for file in files:
                if file.endswith('.jpg'):
                    files_to_move.append(os.path.join(root, file))
    else:
        interesting_files = list(set(files) & set(os.listdir(src)))
        for file in interesting_files:
            files_to_move.append(os.path.join(src, file))
    for file in files_to_move:
        shutil.move(file, dest)

In [4]:
clean_up(True)
rf = Roboflow(api_key="PYZngL70ijcagLdXpb6l")
project = rf.workspace("material-identification").project("garbage-classification-3")
dataset = project.version(2).download("tensorflow")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to GARBAGE-CLASSIFICATION-3-2 in tensorflow:: 100%|██████████| 10472/10472 [00:01<00:00, 10141.41it/s]


In [5]:
train = pd.read_csv('GARBAGE-CLASSIFICATION-3-2/train/_annotations.csv')
test = pd.read_csv('GARBAGE-CLASSIFICATION-3-2/test/_annotations.csv')
val = pd.read_csv('GARBAGE-CLASSIFICATION-3-2/valid/_annotations.csv')
combined_data = pd.concat([train, test, val], ignore_index=True)

In [6]:
move_files_recursively('GARBAGE-CLASSIFICATION-3-2')

In [7]:
map_classes = {
    'METAL': 'METAL_CARDBOARD_PLASTIC',
    'CARDBOARD': 'METAL_CARDBOARD_PLASTIC',
    'PLASTIC': 'METAL_CARDBOARD_PLASTIC',
    'PAPER': 'PAPER',
    'GLASS': 'GLASS',
    'BIODEGRADABLE': 'BIODEGRADABLE'
}
def get_class(c):
    for key in map_classes.keys():
        if key==c:
            return map_classes.get(key)
    return c

def remove_until_char(input_string,remove_char,include_char=True):
    match = re.search(remove_char, input_string)
    if match:
        return input_string[match.start() + 1 if include_char else 0:]
    return input_string

def get_set_name(ratios=None):
    if ratios is None:
        ratios = [0.6, 0.25, 0.15]
    rand = np.random.uniform()
    if rand <= ratios[0]:
        set_name = 'TRAIN'
    elif rand <= ratios[0] + ratios[1]:
        set_name = 'TEST'
    else:
        set_name = 'VAL'
    return set_name

def edit_row(row):
    row['class'] = get_class(row['class'])
    return row

for index, row in combined_data.iterrows():
    combined_data.loc[index] = edit_row(row)

classes_occurences = combined_data['class'].value_counts()    
classes_occurences

class
BIODEGRADABLE              45407
METAL_CARDBOARD_PLASTIC    16484
GLASS                       7809
PAPER                       4390
Name: count, dtype: int64

In [15]:
ratios = [0.6, 0.25, 0.15]
cleansed_data = pd.DataFrame(columns=['set','filename','label','xmin_relative','ymin_relative','1','2','xmax_relative','ymax_relative','3','4'])
for index, row in combined_data.iterrows():
    width = row['width']
    height = row['height']
    row['label']=row['class']
    row['xmin_relative'] = row['xmin'] / width
    row['ymin_relative'] = row['ymin'] / height
    row['xmax_relative'] = row['xmax'] / width
    row['ymax_relative'] = row['ymax'] / height
    row['set'] = get_set_name(ratios)
    row['1'] = None
    row['2'] = None
    row['3'] = None
    row['4'] = None
    cleansed_data.loc[index] = row[['set','filename','label','xmin_relative','ymin_relative','1','2','xmax_relative','ymax_relative','3','4']]
    if index % 500 == 0:
        print(f'Processed {index+1}/{combined_data.shape[0]}')
cleansed_data.to_csv('annotations_full.csv', index=False)

Processed 1/74090
Processed 501/74090
Processed 1001/74090
Processed 1501/74090


KeyboardInterrupt: 

## Balancing data 

In [9]:
cleansed_data = pd.read_csv('annotations_full.csv')
classes_occurences = cleansed_data['label'].value_counts()
classes_occurences

label
BIODEGRADABLE              45407
METAL_CARDBOARD_PLASTIC    16484
GLASS                       7809
PAPER                       4390
Name: count, dtype: int64

In [10]:
balanced_data = pd.DataFrame(columns=cleansed_data.columns)
for class_label in cleansed_data['label'].unique():
    class_samples = cleansed_data[cleansed_data['label'] == class_label].head(np.min(classes_occurences))
    balanced_data = pd.concat([balanced_data, class_samples], ignore_index=True)
print(f'Balanced dataframe shape: {balanced_data.shape}')
balanced_data = balanced_data.sample(frac=1).reset_index(drop=True)
print(f'Balanced dataframe shape shuffled: {balanced_data.shape}')

Balanced dataframe shape: (17560, 11)
Balanced dataframe shape shuffled: (17560, 11)


The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.


In [11]:
balanced_data.to_csv('data/annotations.csv', index=False)
balanced_data

,set,filename,label,xmin_relative,ymin_relative,1,2,xmax_relative,ymax_relative,3,4
0,TRAIN,glass2069_jpg.rf.5057efdda19b00a75c86299d20900...,GLASS,0.100962,0.348558,NaN,NaN,0.846154,0.997596,NaN,NaN
1,TRAIN,paper2072_jpg.rf.26ede102575f09e0f66626711324e...,PAPER,0.000000,0.019231,NaN,NaN,1.000000,0.997596,NaN,NaN
2,TRAIN,biodegradable1233_jpg.rf.16020684434938484f31b...,BIODEGRADABLE,0.067308,0.771635,NaN,NaN,0.129808,0.872596,NaN,NaN
3,TRAIN,biodegradable33_jpg.rf.1872049d97fa8dd9228b0fa...,BIODEGRADABLE,0.141827,0.524038,NaN,NaN,0.992788,0.923077,NaN,NaN
4,TRAIN,glass2396_jpg.rf.3511c6cfd67e9ecdb63bd08d5bb88...,METAL_CARDBOARD_PLASTIC,0.384615,0.016827,NaN,NaN,0.598558,0.079327,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
17555,TRAIN,glass2477_jpg.rf.262d68a59da2cd7a6f860b929c479...,GLASS,0.646635,0.656250,NaN,NaN,0.670673,0.675481,NaN,NaN
17556,TEST,glass3026_jpg.rf.09a244327bd7d9ae333b40023787a...,GLASS,0.608173,0.432692,NaN,NaN,0.680288,0.574519,NaN,NaN
17557,TRAIN,metal627_jpg.rf.22754fe89644bf92291f290b81d09d...,METAL_CARDBOARD_PLASTIC,0.139423,0.002404,NaN,NaN,0.795673,0.574519,NaN,NaN
17558,TEST,paper1408_jpg.rf.9b0a2287af91fba29b8631bc5b303...,PAPER,0.000000,0.117788,NaN,NaN,0.855769,0.713942,NaN,NaN
